In [1]:
import os
import pandas as pd
import tensorflow as tf
import numpy as np
import sys
import time
from sklearn.metrics import accuracy_score
from tqdm import tqdm_notebook as tqdm

sys.path.insert(0, "../../")

from rcalgo.tf.models.semi_supervised.uda import UDA, UDAData
from rcalgo.tf.utils.tokenizers import CharTokenizer
from rcalgo.tf.models.classifier.text_classifier import TextCNN
from rcalgo.tf.training.hook import TensorInfoPrintHook
from rcalgo.tf.training.data.data_iterator import DataIterator

In [2]:
# load data
data_dir = "/home/web_server/antispam/project/datasets/female_vulgar/"
train_df = pd.read_csv(data_dir + 'train_400.csv')
test_df = pd.read_csv(data_dir + 'test.csv')
# count word dict
tokenizer = CharTokenizer.build_from_corpus(test_df['text'], freq_threshold=2)
word_dict = tokenizer.word2idx

In [3]:
# augment unsupvised data with EDA
# warning: this step may be time-costing

org_unsup_file = data_dir + "unsup0.txt"
aug_unsup_file = data_dir + "unsup1.txt"

if not os.path.exists(aug_unsup_file):
    from rcalgo.tf.models.augmentation.token_level.eda import EDA
    eda = EDA(stopwords=[], need_cut=True, rand_seed=0)
    start_t = time.time()
    with open(org_unsup_file) as in_f, open(aug_unsup_file, "w") as out_f:
        sentences = [line.strip() for line in in_f]
        aug_sentences = eda.run(sentences, alpha_sr=0.1, alpha_ri=0.1, alpha_rs=0.1, p_rd=0.1, times=1)
        for line in aug_sentences:
            out_f.write(f"{line}\n")

    print(f"Unsup data augment time cost: {time.time() - start_t} s")

In [4]:
sup_texts = train_df['text']
labels = train_df['label']
batch_size = 32
unsup_ratio = 9
gpu_num = 2
num_epochs = 220
num_train_steps = num_epochs * (int(len(sup_texts)) // (batch_size * gpu_num))
print(f"num_train_steps: {num_train_steps}")

num_train_steps: 1320


In [5]:
# create execute model
config = {
    # uda config
    "unsup_coeff": 1.5,  # unsup loss coeff 
    "tsa_schedule": None,  # linear_schedule, exp_schedule, log_schedule, None
    "batch_size": batch_size,
    "unsup_ratio": unsup_ratio,  # unsup data ratio
    "num_train_steps": num_train_steps - 200,  # num of train steps is needed for tas threshold calcuation
    "uda_softmax_temp": -1,
    "uda_confidence_thresh": -1,
    
    # model config
    "learning_rate": 0.003,

    "num_classes": 2,
    "max_seq_len": 100,
    "embedding_size": 128,
    "num_layers": 2,
    "conv_num_filters": 256,
    "keep_prob": 0.6,
    
    "gpu_id": 0,
    "gpu_num": gpu_num,

    "model_output_dir": "./saved_models",
    "model_name": "cnn"
}

In [6]:
test_sup_texts = test_df['text']
test_labels = test_df['label']

In [7]:
org_unsup_texts = []
aug_unsup_texts = []
with open(org_unsup_file) as org_f:
    all_org_unsup_texts = [line.strip() for line in org_f.readlines()]
with open(aug_unsup_file) as aug_f:
    all_aug_unsup_texts = [line.strip() for line in aug_f.readlines()]

unsup_size = len(all_org_unsup_texts) // unsup_ratio
for i in range(unsup_size):
    org_unsup_texts.append([all_org_unsup_texts[i * unsup_ratio + j] for j in range(unsup_ratio)])
    aug_unsup_texts.append([all_aug_unsup_texts[i * unsup_ratio + j] for j in range(unsup_ratio)])
    
test_size = len(test_labels)
test_org_unsup_texts = org_unsup_texts[:test_size]
test_aug_unsup_texts = aug_unsup_texts[:test_size]

In [8]:
print(f"sup_texts: ")
print(sup_texts[:5])
print(f"org_unsup_texts: ")
print(org_unsup_texts[:5])
print(len(org_unsup_texts))
print(f"aug_unsup_texts: ")
print(aug_unsup_texts[:5])
print(len(test_aug_unsup_texts))

sup_texts: 
0    祝雅儿新年快乐，开心快乐每一天，越来越漂亮哈[笑哭]
1                           非洲哥
2                         鬼街在哪啊
3                            哈卵
4                   差那点洗脸洗手的时间？
Name: text, dtype: object
org_unsup_texts: 
[['祝雅儿新年快乐，开心快乐每一天，越来越漂亮哈[笑哭]', '非洲哥', '鬼街在哪啊', '哈卵', '差那点洗脸洗手的时间？', '?????😻😔🤒👏👨\u200d', '别动我要给你舔', '爱死宝贝师虎', '兜兜转转还是你🤘'], ['改革开放四十周年，吃不完穿不完的。真好！', '今天我湿答答的从你身边经过', '纹身吗😂', '那就祝贺你余生灌满浆，走路拉汤汤，', '快来找我我要馋死了[大哭]', '只有我看见四哥脖子上的草莓了么', '吟诗一首，黑草丛中一刀伤，不用浇水自带汤。两片红唇分大小，一根黄瓜塞满缸。站立樱花闭，蹲时牡丹开，动时双峰舞，用时水成灾，[偷笑]', '我以为是带了个尿不湿呢', '够玩够用'], ['这个姿势有奖励', '老婆让谁给货货大了', '你可还记得我啊？', '🐔吧样', '末将请求一战，如不出水，臣提头来见😀😂😂', '别人怎么说你的', '顺风车欢迎你', '希望你能从校服到婚纱', '要是叫的声音销魂我也就三四分，要是不喜欢能半个小时左右'], ['我会爱你到最后一刻', '山高地肥，费力费牛', '小妹妹好肥啊', '二哥可以打黄油了', '不照也', '希望你张开双腿的时候可以想到我。', '俺娘要', '在酒店和谁开房呢', '冷吗[抱抱][抱抱]'], ['之前路很窄，走的人多了自然宽啦！', '还是会分的看好了。', '大早上的反正我硬了', '我不想长高我只想我老二涨长5厘米😂😂', '没人看到点吗？', '我给七次', '疼疼一疼疼[捂脸]', '后面那小伙看直了哈哈', '也没那么大呀']]
44444
aug_unsup_texts: 
[['祝雅儿新年快乐，开心快乐每虉天，越逨越漂亮哈[笑哭]', '狒洲哥', '鬼劫在哪啊', '哈灤', '差貀点洗亷洗収的时间？

## training with original model

In [9]:
tf.compat.v1.reset_default_graph()
model = TextCNN(config, word_dict, name="default", distribute="tf_distribute")
model.build_model()
model.model_summary()

2020-11-18 10:26:37,396  model.py (73) (MainThread) : INFO  config: {'unsup_coeff': 1.5, 'tsa_schedule': None, 'batch_size': 32, 'unsup_ratio': 10, 'num_train_steps': 1120, 'uda_softmax_temp': -1, 'uda_confidence_thresh': -1, 'learning_rate': 0.003, 'num_classes': 2, 'max_seq_len': 100, 'embedding_size': 128, 'num_layers': 2, 'conv_num_filters': 256, 'keep_prob': 0.5, 'gpu_id': 0, 'gpu_num': 2, 'model_output_dir': './saved_models', 'model_name': 'cnn'}


Instructions for updating:
reduction_indices is deprecated, use axis instead

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

total parameter number: 581378


variable_name variable_shape parameters
0    embedding_layer/embedding_table:0    [3765, 128]     481920
1  my_conv_no.1/conv_1d/he_uniform/W:0  [1, 128, 128]      16384
2             my_conv_no.1/conv_1d/B:0          [128]        128
3  my_conv_no.2/conv_1d/he_uniform/W:0  [2, 128, 128]      32768
4             my_conv_no.2/conv_1d/B:0          [128]        128
5  my_conv_no.3/conv_1d/he_uniform/W:0  [3, 128, 128]      49152
6             my_conv_no.3/conv_1d/B:0          [128]        128
7                  fully_connected/W:0       [384, 2]        768
8                  fully_connected/B:0            [2]          2

In [10]:
model.train_and_evaluate([sup_texts, labels], [test_sup_texts, test_labels], 
            batch_size=batch_size * config['gpu_num'],
            epochs=num_epochs,
            hooks=[])

2020-11-17 19:32:31,771  single_worker_graph_copy_trainer.py (92) (MainThread) : INFO  Initializing trainer ...
2020-11-17 19:32:31,771  single_worker_graph_copy_trainer.py (105) (MainThread) : INFO  gpu_num: 2
2020-11-17 19:32:31,773  graph_copy.py (424) (MainThread) : INFO  build_train_graph kwargs: {'unsup_coeff': 2, 'tsa_schedule': None, 'batch_size': 32, 'unsup_ratio': 3, 'num_train_steps': 1120, 'uda_softmax_temp': -1, 'uda_confidence_thresh': 0.7, 'learning_rate': 0.003, 'num_classes': 2, 'max_seq_len': 100, 'embedding_size': 128, 'num_layers': 2, 'conv_num_filters': 256, 'keep_prob': 0.5, 'model_output_dir': './saved_models', 'model_name': 'cnn', 'excluded_task_set': ['pretrained_model/default'], 'pretrained_model_scope': 'pretrained_model'}
2020-11-17 19:32:31,773  graph_copy.py (426) (MainThread) : INFO  build_train_graph: reconstructing variables...



Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


2020-11-17 19:32:34,038  graph_copy.py (448) (MainThread) : INFO  build_train_graph (default): building inputs...
2020-11-17 19:32:34,038  graph_copy.py (450) (MainThread) : INFO  task: default, <class 'str'>
2020-11-17 19:32:34,043  graph_copy.py (478) (MainThread) : INFO  build_train_graph (default): building per replica graph...



INFO:tensorflow:batch_all_reduce: 9 all-reduces with algorithm = nccl, num_packs = 1, agg_small_grads_max_bytes = 0 and agg_small_grads_max_group = 10


2020-11-17 19:32:35,067  graph_copy.py (486) (MainThread) : INFO  train_ops: [PerReplica:{
  0 /replica:0/task:0/device:GPU:0: <tf.Tensor 'Adam_default/Identity:0' shape=() dtype=int64>,
  1 /replica:0/task:0/device:GPU:1: <tf.Tensor 'Adam_default/Identity_1:0' shape=() dtype=int64>
}], losses: [PerReplica:{
  0 /replica:0/task:0/device:GPU:0: <tf.Tensor 'Mean:0' shape=() dtype=float32>,
  1 /replica:0/task:0/device:GPU:1: <tf.Tensor 'replica_1/Mean:0' shape=() dtype=float32>
}], metrics: [PerReplica:{
  0 /replica:0/task:0/device:GPU:0: <tf.Tensor 'Cast_1:0' shape=(?,) dtype=float32>,
  1 /replica:0/task:0/device:GPU:1: <tf.Tensor 'replica_1/Cast_1:0' shape=(?,) dtype=float32>
}]


...

task:default, lr:3.0000e-03, loss:1.6119, metric:0.6562:   8%|▊         | 47/625 [00:00<00:01, 469.65it/s] 

'epoch 217 train time(sec): 0.028902769088745117'
{'best_train_metrics': [1.0000],
 'best_validate_metrics': [0.6396],
 'epoch_index': 217,
 'train_loss': [0.0002],
 'train_metrics': [1.0000],
 'validate_loss': [1.7053],
 'validate_metrics': [0.6381]}


task:default, lr:3.0000e-03, loss:2.6497, metric:0.5625:   8%|▊         | 48/625 [00:00<00:01, 471.46it/s] 

'epoch 218 train time(sec): 0.02800464630126953'
{'best_train_metrics': [1.0000],
 'best_validate_metrics': [0.6396],
 'epoch_index': 218,
 'train_loss': [0.0000],
 'train_metrics': [1.0000],
 'validate_loss': [1.7301],
 'validate_metrics': [0.6375]}


'epoch 219 train time(sec): 0.027980566024780273'
{'best_train_metrics': [1.0000],
 'best_validate_metrics': [0.6396],
 'epoch_index': 219,
 'train_loss': [0.0001],
 'train_metrics': [1.0000],
 'validate_loss': [1.7399],
 'validate_metrics': [0.6366]}


## training with UDA wrapper

In [9]:
tf.compat.v1.reset_default_graph()
model = TextCNN(config, word_dict, name="default", distribute="tf_distribute")
uda_model = UDA(config=config, execute_model=model, name="default", distribute="tf_distribute")

2020-11-18 16:33:05,182  model.py (73) (MainThread) : INFO  config: {'unsup_coeff': 1.5, 'tsa_schedule': None, 'batch_size': 32, 'unsup_ratio': 9, 'num_train_steps': 1120, 'uda_softmax_temp': -1, 'uda_confidence_thresh': -1, 'learning_rate': 0.003, 'num_classes': 2, 'max_seq_len': 100, 'embedding_size': 128, 'num_layers': 2, 'conv_num_filters': 256, 'keep_prob': 0.6, 'gpu_id': 0, 'gpu_num': 2, 'model_output_dir': './saved_models', 'model_name': 'cnn'}


2020-11-18 16:33:05,205  model.py (73) (MainThread) : INFO  config: {'unsup_coeff': 1.5, 'tsa_schedule': None, 'batch_size': 32, 'unsup_ratio': 9, 'num_train_steps': 1120, 'uda_softmax_temp': -1, 'uda_confidence_thresh': -1, 'learning_rate': 0.003, 'num_classes': 2, 'max_seq_len': 100, 'embedding_size': 128, 'num_layers': 2, 'conv_num_filters': 256, 'keep_prob': 0.6, 'gpu_id': 0, 'gpu_num': 2, 'model_output_dir': './saved_models', 'model_name': 'cnn'}


In [10]:
uda_model.build_model()

print(uda_model.loss)
uda_model.model_summary()


Instructions for updating:
reduction_indices is deprecated, use axis instead


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Tensor("semi-sup-loss:0", shape=(), dtype=float32)
total parameter number: 581378


variable_name variable_shape parameters
0    embedding_layer/embedding_table:0    [3765, 128]     481920
1  my_conv_no.1/conv_1d/he_uniform/W:0  [1, 128, 128]      16384
2             my_conv_no.1/conv_1d/B:0          [128]        128
3  my_conv_no.2/conv_1d/he_uniform/W:0  [2, 128, 128]      32768
4             my_conv_no.2/conv_1d/B:0          [128]        128
5  my_conv_no.3/conv_1d/he_uniform/W:0  [3, 128, 128]      49152
6             my_conv_no.3/conv_1d/B:0          [128]        128
7                  fully_connected/W:0       [384, 2]        768
8                  fully_connected/B:0            [2]          2

In [11]:
logging_hook = TensorInfoPrintHook({"unsup_loss": uda_model.unsup_loss,
#                                     "tsa_threshold": uda_model.tsa_threshold, 
                                    "sup_loss": uda_model.sup_loss})

In [12]:
train_data = UDAData([sup_texts, org_unsup_texts, aug_unsup_texts, labels],
                     mode="train", batch_size=batch_size * config['gpu_num'], 
                     drop_remainder=True, name="train_data")

test_data = UDAData([test_sup_texts, test_org_unsup_texts, test_aug_unsup_texts, test_labels],
                     mode="test", batch_size=batch_size * config['gpu_num'], 
                    drop_remainder=True, name="test_data")

2020-11-18 16:33:05,765  data.py (19) (MainThread) : INFO  train_data: will drop 15 remainder_size data items.
2020-11-18 16:33:06,367  data.py (19) (MainThread) : INFO  test_data: will drop 0 remainder_size data items.


In [13]:
print(len(sup_texts), len(org_unsup_texts), len(aug_unsup_texts), len(labels))
print(len(test_sup_texts), len(test_org_unsup_texts), len(test_aug_unsup_texts), len(test_labels))

399 44444 44444 399
40000 40000 40000 40000


In [ ]:
uda_model.train_and_evaluate(train_data, test_data,
                batch_size=batch_size * config['gpu_num'],
                epochs=num_epochs,
                hooks=[logging_hook])

2020-11-18 16:33:06,923  single_worker_graph_copy_trainer.py (92) (MainThread) : INFO  Initializing trainer ...
2020-11-18 16:33:06,924  single_worker_graph_copy_trainer.py (105) (MainThread) : INFO  gpu_num: 2
2020-11-18 16:33:06,925  graph_copy.py (424) (MainThread) : INFO  build_train_graph kwargs: {'unsup_coeff': 1.5, 'tsa_schedule': None, 'batch_size': 32, 'unsup_ratio': 9, 'num_train_steps': 1120, 'uda_softmax_temp': -1, 'uda_confidence_thresh': -1, 'learning_rate': 0.003, 'num_classes': 2, 'max_seq_len': 100, 'embedding_size': 128, 'num_layers': 2, 'conv_num_filters': 256, 'keep_prob': 0.6, 'model_output_dir': './saved_models', 'model_name': 'cnn', 'excluded_task_set': ['pretrained_model/default'], 'pretrained_model_scope': 'pretrained_model'}
2020-11-18 16:33:06,926  graph_copy.py (426) (MainThread) : INFO  build_train_graph: reconstructing variables...



Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


2020-11-18 16:33:09,602  graph_copy.py (448) (MainThread) : INFO  build_train_graph (default): building inputs...
2020-11-18 16:33:09,602  graph_copy.py (450) (MainThread) : INFO  task: default, <class 'str'>
2020-11-18 16:33:09,608  graph_copy.py (478) (MainThread) : INFO  build_train_graph (default): building per replica graph...



INFO:tensorflow:batch_all_reduce: 9 all-reduces with algorithm = nccl, num_packs = 1, agg_small_grads_max_bytes = 0 and agg_small_grads_max_group = 10


2020-11-18 16:33:10,951  graph_copy.py (486) (MainThread) : INFO  train_ops: [PerReplica:{
  0 /replica:0/task:0/device:GPU:0: <tf.Tensor 'Adam_default/Identity:0' shape=() dtype=int64>,
  1 /replica:0/task:0/device:GPU:1: <tf.Tensor 'Adam_default/Identity_1:0' shape=() dtype=int64>
}], losses: [PerReplica:{
  0 /replica:0/task:0/device:GPU:0: <tf.Tensor 'semi-sup-loss:0' shape=() dtype=float32>,
  1 /replica:0/task:0/device:GPU:1: <tf.Tensor 'replica_1/semi-sup-loss:0' shape=() dtype=float32>
}], metrics: [PerReplica:{
  0 /replica:0/task:0/device:GPU:0: <tf.Tensor 'Cast_1:0' shape=(?,) dtype=float32>,
  1 /replica:0/task:0/device:GPU:1: <tf.Tensor 'replica_1/Cast_1:0' shape=(?,) dtype=float32>
}]


...

  0%|          | 0/6 [00:00<?, ?it/s]                                                                      

last: {'unsup_loss': 0.09678134, 'sup_loss': 0.00588357}
'epoch 171 train time(sec): 0.11306953430175781'
{'best_train_metrics': [1.0000],
 'best_validate_metrics': [0.7270],
 'epoch_index': 171,
 'train_loss': [0.1279],
 'train_metrics': [1.0000],
 'validate_loss': [0.7408],
 'validate_metrics': [0.7155]}


last: {'unsup_loss': 0.098289646, 'sup_loss': 0.008585375}


'epoch 172 train time(sec): 0.11379718780517578'
{'best_train_metrics': [1.0000],
 'best_validate_metrics': [0.7270],
 'epoch_index': 172,
 'train_loss': [0.1289],
 'train_metrics': [1.0000],
 'validate_loss': [0.7298],
 'validate_metrics': [0.7197]}


task:default, lr:3.0000e-03, loss:0.7447, metric:0.6875:  28%|██▊       | 172/625 [00:01<00:03, 126.78it/s]

In my experiment, with 400 training data of `female_vulgar`, the validate accuracy improved from **0.6458** to **0.7283**.

Suggestions: 

- Keep the training steps long enough. Only then could we learn enough from the unsupervised data, which is huge compare to supervised data.
- Data augmentation is important. Try more augmentation methods if you found one not worked.
- Turn on the tsa-schdule if the supervised data is small. Otherwise you can turn it off.

## Export and infer

In [17]:
uda_model.export_freeze_graph("freeze", 1, multi_replica_ckpt_path="saved_models/best_model/default-epoch177.ckpt")

2020-11-18 10:50:33,442  single_worker_graph_copy_trainer.py (431) (MainThread) : INFO  Restore 9 vars from checkpoint.



Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
INFO:tensorflow:Froze 9 variables.
INFO:tensorflow:Converted 9 variables to const ops.


In [18]:
def load_feeze_graph(model_fn):
    gpu_options = tf.compat.v1.GPUOptions(allow_growth=True)
    config = tf.compat.v1.ConfigProto(gpu_options=gpu_options, allow_soft_placement=True)
    session = tf.compat.v1.Session(config=config, graph=tf.Graph())
    with session.graph.as_default():
        tf.compat.v1.train.import_meta_graph(model_fn)
        session.run(tf.compat.v1.tables_initializer())
    return session

sess = load_feeze_graph('freeze/1/frozen.pb')

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [19]:
with sess.graph.as_default():
    inputs = tf.get_collection('input_dict')
    outputs = tf.get_collection('output_dict')[0]

    data_iter = DataIterator([test_sup_texts], 
                             np.array(range(len(test_sup_texts))), batch_size=batch_size)
    data_iter_tqdm = tqdm(data_iter, leave=False)
    res = []
    for idx, batch in data_iter_tqdm:
        feed_data = dict(zip(inputs, batch))
        batch_res = sess.run(outputs, feed_dict=feed_data)
        res.append(batch_res)
       
    np_res = np.concatenate(res, axis=0)
    
print(np_res)

/root/anaconda3/envs/tf1.15/lib/python3.6/site-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys


[[0.77381283 0.22618717]
 [0.87828565 0.12171435]
 [0.05804914 0.9419508 ]
 ...
 [0.9693536  0.03064639]
 [0.0588976  0.9411023 ]
 [0.96074843 0.03925158]]


In [20]:
pos_predicts = np.argmax(np_res, axis=1)
pos_acc = accuracy_score(test_labels, pos_predicts)
print(f"acc on test: {pos_acc}")

acc on test: 0.71915
